In [1]:
import tensorflow as tf
import pickle

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from glob import glob
from PIL import Image
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
train_data_generator= ImageDataGenerator(rescale = 1./255,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True)

import pydicom
import scipy
import cv2
import skimage



In [2]:
training_data_path = '/Users/sipebradford/Downloads/Covid-19-detection-from-chest-xray/xray_dataset_covid19/train'
test_data_path = '/Users/sipebradford/Downloads/Covid-19-detection-from-chest-xray/xray_dataset_covid19/test'

output_classes = len(glob(training_data_path + '/*'))
#print('Number of output classes is: ', output_classes)

training_files = glob(training_data_path + '/*/*')
test_files = glob(test_data_path + '/*/*')
#print(len(training_files))
#print(len(test_files))

test_image = Image.open(training_files[0])
test_image = test_image.resize((400,400))

##plt.imshow(test_image)
##plt.show()

test_data_generator = ImageDataGenerator(rescale = 1./255)

training_data = train_data_generator.flow_from_directory(training_data_path, target_size = (400,400), batch_size = 4, class_mode = 'binary')

test_data = train_data_generator.flow_from_directory(test_data_path, target_size = (400,400), batch_size = 4, class_mode = 'binary')
input_layer = Input(shape = (400,400,3))
conv1 = Conv2D(32, (3,3), strides = 2, activation = 'relu')(input_layer)
maxpool1 = MaxPool2D(2,2)(conv1)
conv2 = Conv2D(64, (3,3), strides = 2, activation = 'relu')(maxpool1)
maxpool2 = MaxPool2D(pool_size = (2, 2))(conv2)
flat1 = Flatten()(maxpool2)
dense1 = Dense(256, activation = 'relu')(flat1)
output_layer = Dense(1, activation = 'sigmoid')(dense1)
model = Model(input_layer, output_layer)

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


2022-02-28 10:39:17.533870: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#compiling the CNN model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_history = model.fit(training_data, epochs = 12, validation_data = test_data, verbose = 1)
filename = 'finalized_model2.sav'
pickle.dump(model, open(filename, 'wb'))

Epoch 1/12
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
37/37 [==============================] - ETA: 0s - loss: 0.7982 - accuracy: 0.5946WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fc3f3cce7a0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and 